## Step 1: Import Packages and Declare Constants

In [106]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [107]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

In [108]:
#Replace this value with the S3 Bucket Created
default_bucket = "amazon-sagemaker-438465168169-us-east-1-e9eb0fb68840"

In [109]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


## Step 2: Generate Baseline Dataset

Baseline Data will be used as part of SageMaker Clarify Step to generate SHAP Values 

In [110]:
def preprocess_data(file_path):
    df = pd.read_excel(file_path)
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    return df

In [111]:
baseline_data = preprocess_data("storedata_total.xlsx")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

In [112]:
pd.DataFrame(baseline_sample).to_csv("baseline.csv",header=False,index=False)

## Step 3: Generate Batch Dataset

In [113]:
batch_data = preprocess_data("storedata_total.xlsx")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

In [114]:
pd.DataFrame(batch_sample).to_csv("batch.csv",header=False,index=False)

## Step 4: Copy Data and Scripts to S3 Bucket

In [115]:
df = pd.read_excel("storedata_total.xlsx")
df.to_csv("storedata_total.csv", index=False)

In [116]:
s3_client = boto3.resource('s3')
s3_client.Bucket(default_bucket).upload_file("storedata_total.csv","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/data/storedata_total.csv")
s3_client.Bucket(default_bucket).upload_file("batch.csv","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/data/batch/batch.csv")
s3_client.Bucket(default_bucket).upload_file("baseline.csv","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/input/baseline/baseline.csv")

In [117]:
s3_client.Bucket(default_bucket).upload_file("pipelines/preprocess.py","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/evaluate.py","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/generate_config.py","dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/input/code/generate_config.py")

## Step 5: Get the Pipeline Instance

In [118]:
from pipelines.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=f"{default_bucket}/dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev",
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Ap

In [119]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.c4.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.c4.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://amazon-sagemaker-438465168169-us-east-1-e9eb0fb68840/dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://amazon-sagemaker-438465168169-us-east-1-e9eb0fb68840/dzd_4gqe4c7fmeaa5j/dey1k5789rqn1j/dev/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"G

## Step 5: Submit the pipeline to SageMaker and start execution

In [120]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:438465168169:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '92dd174e-0b4a-48ed-a15c-c197bc1a909f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '92dd174e-0b4a-48ed-a15c-c197bc1a909f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Fri, 14 Feb 2025 15:02:44 GMT'},
  'RetryAttempts': 0}}

Start Pipeline Execution

In [121]:
execution = pipeline.start()

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [144]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:438465168169:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:438465168169:pipeline/ChurnModelSMPipeline/execution/858kbwryna9p',
 'PipelineExecutionDisplayName': 'execution-1739545364790',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'churnmodelsmpipeline',
  'TrialName': '858kbwryna9p'},
 'CreationTime': datetime.datetime(2025, 2, 14, 15, 2, 44, 680000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 14, 15, 2, 44, 680000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:438465168169:user-profile/d-69qrstv0krfc/e37b481b-6b4c-4419-848a-2a0a5995e4a1',
  'UserProfileName': 'e37b481b-6b4c-4419-848a-2a0a5995e4a1',
  'DomainId': 'd-69qrstv0krfc',
  'IamIdentity': {'Arn': 'arn:aws:sts::438465168169:assumed-role/datazone_usr_role_dey1k5789rqn1j_6lz335u9nebcx3/SageMaker',
   'PrincipalId': 'AROAWMFUPQMUQ5FX4ZXVN:SageMa

We can list the execution steps to check out the status and artifacts:

In [145]:
execution.list_steps()

[{'StepName': 'ChurnModelConfigFile',
  'StartTime': datetime.datetime(2025, 2, 14, 15, 11, 51, 142000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:438465168169:processing-job/pipelines-858kbwryna9p-ChurnModelConfigFile-ymyiO46N7u'}},
  'AttemptCount': 1},
 {'StepName': 'ChurnTransform',
  'StartTime': datetime.datetime(2025, 2, 14, 15, 11, 51, 142000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 2, 14, 15, 11, 53, 101000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': "ClientError: Failed to invoke sagemaker:CreateTransformJob. Error Details: The account-level service limit 'ml.c4.xlarge for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.\nRetry not appropr